In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-2bcc7f3a-e041-1513-833d-f3e27896e6d5)


In [2]:
!ls

Firm_relation_extraction_LUKE.ipynb  lexisnexis
LUKE_model.py			     lexisnexis_for_inference
__pycache__			     utils.py
lexis_entity_extraction_spacy.ipynb  wandb


In [3]:
# !ls LUKE

In [4]:
!rm -r LUKE # delete old runs

rm: cannot remove 'LUKE': No such file or directory


In this notebook, we are going to fine-tune [`LukeForEntityPairClassification`](https://huggingface.co/transformers/model_doc/luke.html#lukeforentitypairclassification) on a supervised **relation extraction** dataset.

The goal for the model is to predict, given a sentence and the character spans of two entities within the sentence, the relationship between the entities.

* Paper: https://arxiv.org/abs/2010.01057
* Original repository: https://github.com/studio-ousia/luke

In [5]:
!pip install -q transformers pandas sklearn pytorch-lightning wandb unidecode

In [6]:
from LUKE_model import RelationExtractionDataset, LUKE
import wandb
import torch
import pytorch_lightning
import pandas as pd

In [7]:
pytorch_lightning.utilities.seed.seed_everything(seed=42)

Global seed set to 42


42

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Read in data

Let's download the data from the web, hosted on Dropbox.

Each row in the dataframe consists of a news article. 

In [9]:
df = pd.read_pickle("https://www.dropbox.com/s/rdwg8d76ytqqgdy/training-data-29-03-22.pkl?dl=1")
df

,Date,source,document,firms,spans,rels
233892045,1985-01-01,Thomson SDC alliances - Deal Number 233892045,Accel Capital L.P. invested an undisclosed amo...,"[Accel Capital, Thalamus Electronics Inc]","[(0, 13), (53, 77)]","[StrategicAlliance, Manufacturing]"
216749045,1985-01-01,Thomson SDC alliances - Deal Number 216749045,Standard Microsystems Corp. and Mitsubishi Ele...,"[Standard Microsystems Corp, Mitsubishi Electr...","[(0, 26), (32, 56)]","[StrategicAlliance, Licensing]"
234987045,1985-01-01,Thomson SDC alliances - Deal Number 234987045,"Elf Technologies, Inc., the venture capital in...","[Elf Technologies, Santa Clara Systems]","[(0, 16), (169, 188)]",[StrategicAlliance]
216600045,1985-01-01,Thomson SDC alliances - Deal Number 216600045,Stauffer Chemical Co. has been licensed by Mit...,"[Stauffer Chemical Co, Himont Inc]","[(0, 20), (79, 89)]","[StrategicAlliance, Marketing, Manufacturing, ..."
2410877045,1985-01-01,Thomson SDC alliances - Deal Number 2410877045,The Romanian Government (RG) and Control Data ...,"[Romania, Control Data Corp]","[(4, 11), (33, 50)]","[JointVenture, Manufacturing]"
...,...,...,...,...,...,...
257823,NaT,Reuters News Dataset - Article ID http://www.b...,University of Notre Dame football radio anal...,"[Notre Dame, Notre Dame’s]","[(15, 25), (200, 212)]",[]
416616,NaT,Reuters News Dataset - Article ID http://www.b...,"Schaeffler AG, the industrial-bearing maker th...","[Schaeffler AG, Continental AG]","[(0, 13), (91, 105)]",[]
328325,NaT,Reuters News Dataset - Article ID http://www.r...,Barnes & Noble Inc ( BKS.N ) said on Friday it...,"[Barnes & Noble, Lynch]","[(0, 14), (113, 118)]",[]
121959,NaT,Reuters News Dataset - Article ID http://www.b...,(Corrects third paragraph to show output of 25...,"[First Solar Inc ., Edison International]","[(154, 171), (249, 269)]",[]


Shuffle the data:

In [10]:
df = df.sample(frac=1)

This is the frequency of our relation types:



In [11]:
df.rels.explode().value_counts()

StrategicAlliance         83845
JointVenture              44556
Pending                   38916
Marketing                 22478
Manufacturing             19926
ResearchandDevelopment    13568
Licensing                 12857
TechnologyTransfer         9889
Supply                     5407
Exploration                3152
Terminated                 1578
Name: rels, dtype: int64

We don't want to consider the less relevant and less consistently applied labels of 'supply', 'exploration', and 'technology transfer'. Let's remove them:

In [12]:
# df['rels'] = df.rels.apply(lambda rels: [rel for rel in rels if rel not in ['TechnologyTransfer', 'Supply', 'Exploration']])

In [13]:
rel_frequencies = df.rels.explode().value_counts()
rel_frequencies

StrategicAlliance         83845
JointVenture              44556
Pending                   38916
Marketing                 22478
Manufacturing             19926
ResearchandDevelopment    13568
Licensing                 12857
TechnologyTransfer         9889
Supply                     5407
Exploration                3152
Terminated                 1578
Name: rels, dtype: int64

Let's one hot encode the relationship classes:

In [14]:
rel_label_names = rel_frequencies.index.to_list()

def label2ids(labels, rel_label_names=rel_label_names):
  return [1 if rel_name in labels else 0 for rel_name in rel_label_names]

df['rel_one_hot'] = df.rels.apply(label2ids)

def ids2labels(ids, rel_label_names=rel_label_names):
  labels = []
  for idx, label in enumerate(rel_label_names):
    if ids[idx] == 1:
      labels.append(label)
    
  return labels

df.sample(10)

,Date,source,document,firms,spans,rels,rel_one_hot
18580,NaT,Reuters News Dataset - Article ID http://www.r...,The UK government has approached energy suppli...,"[British Energy, the Financial Times]","[(131, 145), (163, 182)]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3211258045,2018-01-16,Thomson SDC alliances - Deal Number 3211258045,Bio-On SpA and Hera Servizi Energia Srl formed...,"[Bio-On SpA, Hera Servizi Energia Srl]","[(0, 10), (15, 39)]",[StrategicAlliance],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1808405045,2006-11-01,Thomson SDC alliances - Deal Number 1808405045,"Zynchros Inc (ZI) and Medi-Span Inc (MS), a un...","[Zynchros Inc, Medi-Span Inc]","[(0, 12), (22, 35)]","[StrategicAlliance, TechnologyTransfer]","[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
21619,NaT,Reuters News Dataset - Article ID http://www.r...,U.S. antitrust authorities have given Arby's o...,"[Triarc, Wendy]","[(51, 57), (102, 107)]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3212678045,2014-10-10,Thomson SDC alliances - Deal Number 3212678045,T2 Biosystems Inc and SMC Ltd formed a strateg...,"[T2 Biosystems Inc, SMC Ltd]","[(0, 17), (22, 29)]","[StrategicAlliance, Supply]","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
1629561045,2005-01-05,Thomson SDC alliances - Deal Number 1629561045,Control4 and Tweeter Home Entertainment Group ...,"[Control4, Tweeter Home Ent]","[(0, 8), (13, 29)]","[StrategicAlliance, Marketing]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
3066841045,2017-01-30,Thomson SDC alliances - Deal Number 3066841045,Universities Superannuation Scheme Ltd and Mor...,"[Universities Superannuation, Morgan Sindall I...","[(0, 27), (43, 73)]","[JointVenture, Pending]","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
486782045,1995-08-16,Thomson SDC alliances - Deal Number 486782045,"Mobil Natural Gas Inc, a unit of Mobil Corp, e...","[Mobil Natural Gas, CNB/Olympic Gas Services]","[(0, 17), (84, 108)]","[StrategicAlliance, Marketing]","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
1998281045,2008-07-23,Thomson SDC alliances - Deal Number 1998281045,"Boutet SA, a unit of Rexel SA, and Cadolto Gmb...","[Boutet SA, Cadolto GmbH]","[(0, 9), (35, 47)]",[JointVenture],"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
367837,NaT,Reuters News Dataset - Article ID http://www.b...,"Visteon Corp. (VC) ’s South Korean unit, the w...","[Visteon Corp., Hyundai Motor Co.]","[(0, 13), (206, 223)]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


## Define the PyTorch dataset and dataloaders

Next, we define regular PyTorch datasets and corresponding dataloaders. In PyTorch, you need to define a `Dataset` class that inherits from `torch.utils.data.Dataset`, and you need to implement 3 methods: the `init` method (for initializing the dataset with data), the `len` method (which returns the number of elements in the dataset) and the `getitem()` method, which returns a single item from the dataset.

In our case, each item of the dataset consists of a sentence, the spans of 2 entities in the sentence, and a label of the relationship. We use `LukeTokenizer` (available in the Transformers library) to turn these into the inputs expected by the model, which are `input_ids`, `entity_ids`, `attention_mask`, `entity_attention_mask` and `entity_position_ids`.

For more information regarding these inputs, refer to the [docs](https://huggingface.co/transformers/model_doc/luke.html#lukeforentitypairclassification) of `LukeForEntityPairClassification`.


Let's set our hyperparameters:

In [15]:
MAX_LEN = 128
LEARNING_RATE = 4e-5
BATCH_SIZE = 128
MAX_EPOCHS = 10
THRESHOLDS = torch.tensor([0.5 for rel in rel_label_names]).to(device) # can change this for class-specific thresholds
WEIGHT_DECAY= 0.01
GRAD_CLIP_VAL = 0
CLASS_WEIGHTS = torch.tensor([1 for rel in rel_label_names]).to(device) # equal class weights

Here we instantiate the class defined above with 3 objects: a training dataset, a validation dataset and a test set.

In [16]:
from sklearn.model_selection import train_test_split

train_size = 0.8
validation_size = 0.1
test_size = 0.1

train_df, test_df = train_test_split(df, test_size=1-train_size, shuffle=True)
val_df, test_df = train_test_split(test_df, test_size=test_size/(test_size+validation_size), shuffle=False)

print("FULL Dataset: {}".format(len(df)))
print("TRAIN Dataset: {}".format(len(train_df)))
print("TEST Dataset: {}".format(len(test_df)))
print("VALIDATION Dataset: {}".format(len(val_df)))

# define the dataset
train_dataset = RelationExtractionDataset(data=train_df)
val_dataset = RelationExtractionDataset(data=val_df)
test_dataset = RelationExtractionDataset(data=test_df)

FULL Dataset: 264928
TRAIN Dataset: 211942
TEST Dataset: 26493
VALIDATION Dataset: 26493


In [17]:
train_dataset[0].keys()

dict_keys(['input_ids', 'entity_ids', 'entity_position_ids', 'attention_mask', 'entity_attention_mask', 'label'])

## Define a PyTorch LightningModule

Let's define the model as a PyTorch LightningModule. A `LightningModule` is actually an `nn.Module`, but with some extra functionality.

For more information regarding how to define this, see the [docs](https://pytorch-lightning.readthedocs.io/en/latest/?_ga=2.56317931.1395871250.1622709933-1738348008.1615553774) of PyTorch Lightning.

In [18]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [19]:
print(f'Free CUDA memory: {(torch.cuda.get_device_properties(0).total_memory-torch.cuda.memory_allocated(0))/1e9:.2f}GB')

Free CUDA memory: 25.45GB


In [20]:
model = LUKE(num_labels=len(rel_label_names), lr=LEARNING_RATE, batch_size=BATCH_SIZE, class_weights=CLASS_WEIGHTS,
                 thresholds=THRESHOLDS, weight_decay=WEIGHT_DECAY, label_names=rel_label_names,
                 datasets={'train_dataset': train_dataset, 'val_dataset': val_dataset, 'test_dataset': test_dataset})

Some weights of the model checkpoint at studio-ousia/luke-base were not used when initializing LukeForEntityPairClassification: ['embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LukeForEntityPairClassification were not initialized from the model checkpoint at studio-ousia/luke-base and are newly initialized: ['classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


"batch_size":    128
"class_weights": tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
"datasets":      {'train_dataset': <LUKE_model.RelationExtractionDataset object at 0x7f5534dd4700>, 'val_dataset': <LUKE_model.RelationExtractionDataset object at 0x7f5534de4e80>, 'test_dataset': <LUKE_model.RelationExtractionDataset object at 0x7f56bc3b1b20>}
"label_names":   ['StrategicAlliance', 'JointVenture', 'Pending', 'Marketing', 'Manufacturing', 'ResearchandDevelopment', 'Licensing', 'TechnologyTransfer', 'Supply', 'Exploration', 'Terminated']
"lr":            4e-05
"num_labels":    11
"thresholds":    tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
        0.5000, 0.5000], device='cuda:0')
"weight_decay":  0.01


## Train the model

Let's train the model. We also use early stopping, to avoid overfitting the training dataset. We also log everything to Weights and Biases, which will give us beautiful charts of the loss and accuracy plotted over time.

If you haven't already, you can create an account on the [website](https://wandb.ai/site), then log in in a web browser, and run the cell below: 

In [21]:
wandb.login()

wandb: Currently logged in as: jakr (use `wandb login --relogin` to force relogin)


True

In [22]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

wandb_logger = WandbLogger(project='LUKE')

# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='val_f1_macro_avg',
    patience=3,
    strict=False,
    verbose=True,
    mode='max',
    min_delta=0 # stops training once no more improvement is made
)

checkpoint_callback = ModelCheckpoint(dirpath='LUKE', save_top_k=1, save_last=False, verbose=True,
                                      monitor='val_f1_macro_avg', mode='max', every_n_epochs=1)

trainer = Trainer(gpus=1, logger=wandb_logger, callbacks=[early_stop_callback, checkpoint_callback], 
                  max_epochs=MAX_EPOCHS, precision=16, 
                  stochastic_weight_avg=True, auto_lr_find=True, 
                  benchmark=True, deterministic=True, min_epochs=1,
                  val_check_interval=0.1, gradient_clip_val=GRAD_CLIP_VAL, fast_dev_run=False)
                    # limit_train_batches=0.25

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:146: UserWarning: You passed `deterministic=True` and `benchmark=True`. Note that PyTorch ignores torch.backends.cudnn.deterministic=True when torch.backends.cudnn.benchmark=True.
  rank_zero_warn(
Using 16bit native Automatic Mixed Precision (AMP)
/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:64: LightningDeprecationWarning: Setting `Trainer(stochastic_weight_avg=True)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.stochastic_weight_avg.StochasticWeightAveraging` directly to the Trainer's `callbacks` argument instead.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Finding the optimal learning rate:

In [23]:
# lr_finder = trainer.tuner.lr_find(model)

In [24]:
# lr_finder.suggestion()

Setting the learning rate to the optimal rate:

In [25]:
# model.hparams.lr = lr_finder.suggestion()

Let's train the model:

In [26]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                            | Params
----------------------------------------------------------
0 | model | LukeForEntityPairClassification | 274 M 
----------------------------------------------------------
274 M     Trainable params
0         Non-trainable params
274 M     Total params
549.038   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


Training: 0it [00:00, ?it/s]

2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


...

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


In [32]:
trainer.test()

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1447: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /workspace/lexis_firm_network/LUKE/epoch=0-step=990.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /workspace/lexis_firm_network/LUKE/epoch=0-step=990.ckpt


Testing: 0it [00:00, ?it/s]

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


...

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
             Test metric                         DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test_f1_Exploration                  0.5640326662065795
        test_f1_JointVenture                  0.9849205543493995
          test_f1_Licensing                    0.831468432126562
        test_f1_Manufacturing                 0.8369841359836606
          test_f1_Marketing                   0.7777563948503549
           test_f1_Pending                    0.9453754145118911
   test_f1_ResearchandDevelopment             0.7462652044575501
      test_f1_StrategicAlliance               0.9914315596001315
           test_f1_Supply                     0.43581335465393434
     test_f1_TechnologyTransfer               0.5594585831584981
         test_f1_Terminated                  

[{'test_loss': 0.04837501421570778,
  'test_precision_StrategicAlliance': 0.9876806515317311,
  'test_recall_StrategicAlliance': 0.9954363398888277,
  'test_f1_StrategicAlliance': 0.9914315596001315,
  'test_precision_JointVenture': 0.9919712234320415,
  'test_recall_JointVenture': 0.9786332843123607,
  'test_f1_JointVenture': 0.9849205543493995,
  'test_precision_Pending': 0.9428081298612162,
  'test_recall_Pending': 0.9503574575914892,
  'test_f1_Pending': 0.9453754145118911,
  'test_precision_Marketing': 0.8172933512447577,
  'test_recall_Marketing': 0.7553746555771281,
  'test_f1_Marketing': 0.7777563948503549,
  'test_precision_Manufacturing': 0.8630040975096386,
  'test_recall_Manufacturing': 0.8255537199058086,
  'test_f1_Manufacturing': 0.8369841359836606,
  'test_precision_ResearchandDevelopment': 0.7503370891052048,
  'test_recall_ResearchandDevelopment': 0.7696641991931845,
  'test_f1_ResearchandDevelopment': 0.7462652044575501,
  'test_precision_Licensing': 0.89246397724658

In [33]:
checkpoint_callback.best_model_path

'/workspace/lexis_firm_network/LUKE/epoch=0-step=990.ckpt'

In [29]:
# !ls LUKE

## Evaluation

Instead of calling `trainer.test()`, we can also manually evaluate the model on the entire test set:

In [39]:
outputs = trainer.predict(dataloaders=model.test_dataloader())

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1447: UserWarning: `.predict(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `predict(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /workspace/lexis_firm_network/LUKE/epoch=0-step=990.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /workspace/lexis_firm_network/LUKE/epoch=0-step=990.ckpt


Predicting: 1485it [00:00, ?it/s]

1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
2 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens
1 entities are ignored because their entity spans are invalid due to the truncation of input tokens


## Inference

Here we test the trained model on a new, unseen sentence.

In [46]:
from transformers import LukeTokenizer
import torch.nn.functional as F
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base", task="entity_pair_classification")
model.model.eval()
model.to(device)

LUKE(
  (model): LukeForEntityPairClassification(
    (luke): LukeModel(
      (embeddings): LukeEmbeddings(
        (word_embeddings): Embedding(50267, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (entity_embeddings): LukeEntityEmbeddings(
        (entity_embeddings): Embedding(500000, 256, padding_idx=0)
        (entity_embedding_dense): Linear(in_features=256, out_features=768, bias=False)
        (position_embeddings): Embedding(514, 768)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): LukeEncoder(
        (layer): ModuleList(
          (0): LukeLayer(
            (attention): LukeAttention(
              (

In [59]:
test_doc = test_df[test_df.rels.apply(lambda rels: 'ResearchandDevelopment' in rels)].sample()

text = test_doc.document.iloc[0]
spans = test_doc.spans.iloc[0]  # character-based entity spans 
inputs = tokenizer(text, entity_spans=spans, padding="max_length", truncation=True, return_tensors="pt",
                            max_length=MAX_LEN)
inputs.to(device)

outputs = model.model(**inputs)
logits = outputs.logits
predicted_classes = (torch.sigmoid(logits)>THRESHOLDS).float()
print("Sentence:", text)
print("Tokens:", tokenizer.decode(inputs["input_ids"][0]))
print("Ground truth label:", ids2labels(test_doc.rel_one_hot.iloc[0]))
print("Predicted class idx:", ids2labels(predicted_classes.squeeze().tolist()))
print("Confidence:", torch.sigmoid(logits).squeeze().tolist())

Sentence: PenMagic, Time Warner's Money Magazine division, and Apple Computer agreed to cooperatively develop two financial applications for Apple's Newton personal digital assistant (PDA). PenMagic granted Apple an exclusive license to the Money Magazine Business Forms and Money Magazine Financial Assistant. Apple was expected to market the two applications with all PDAs using the Newton operating system.
Tokens: <s>PenMagic, Time Warner's <ent>  Money Magazine <ent>  division, and <ent2>  Apple Computer <ent2>  agreed to cooperatively develop two financial applications for Apple's Newton personal digital assistant (PDA). PenMagic granted Apple an exclusive license to the Money Magazine Business Forms and Money Magazine Financial Assistant. Apple was expected to market the two applications with all PDAs using the Newton operating system.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><

## Inference on Lexis Nexis news

In [ ]:
# model = LUKE.load_from_checkpoint(checkpoint_path='LUKE/epoch=1-step=2316.ckpt')

# model = model.to(device)

In [40]:
df = pd.read_pickle('lexisnexis_for_inference')
df

,title,document,publication,word_count,publication_date,publication_date_text,author,copyright,subject,country,city,person,industry,company,lang,firms,firms,spans
3,![['ELectricCar News: Kandi's (NASDAQ GS: $KND...,"! Jinhua, China - July 25, 2016 (Newsfile Corp...",['MENA English (Middle East and North Africa F...,4514,2016-07-26,"July 26, 2016 Tuesday",NaN,Copyright 2016 MENAFN.COM All Rights Reserved,"[JOINT VENTURES, ELECTRIC VEHICLES, HOLDING CO...","[CHINA, NORTHERN AFRICA, MIDDLE EAST]","[DONGGUAN, GUANGDONG, CHINA]",[],"[NAICS336211 MOTOR VEHICLE BODY MANUFACTURING,...","[KANDI TECHNOLOGIES GROUP INC, GEELY AUTOMOBIL...",en,"[(Kandi Technologies, (78, 96), kandi technolo...","[Kandi Technologies, Geely Automobile Holdings...","[(78, 96), (274, 303)]"
3,![['ELectricCar News: Kandi's (NASDAQ GS: $KND...,"! Jinhua, China - July 25, 2016 (Newsfile Corp...",['MENA English (Middle East and North Africa F...,4514,2016-07-26,"July 26, 2016 Tuesday",NaN,Copyright 2016 MENAFN.COM All Rights Reserved,"[JOINT VENTURES, ELECTRIC VEHICLES, HOLDING CO...","[CHINA, NORTHERN AFRICA, MIDDLE EAST]","[DONGGUAN, GUANGDONG, CHINA]",[],"[NAICS336211 MOTOR VEHICLE BODY MANUFACTURING,...","[KANDI TECHNOLOGIES GROUP INC, GEELY AUTOMOBIL...",en,"[(Kandi Technologies, (78, 96), kandi technolo...","[Kandi Technologies, GS]","[(78, 96), (117, 119)]"
3,![['ELectricCar News: Kandi's (NASDAQ GS: $KND...,"! Jinhua, China - July 25, 2016 (Newsfile Corp...",['MENA English (Middle East and North Africa F...,4514,2016-07-26,"July 26, 2016 Tuesday",NaN,Copyright 2016 MENAFN.COM All Rights Reserved,"[JOINT VENTURES, ELECTRIC VEHICLES, HOLDING CO...","[CHINA, NORTHERN AFRICA, MIDDLE EAST]","[DONGGUAN, GUANGDONG, CHINA]",[],"[NAICS336211 MOTOR VEHICLE BODY MANUFACTURING,...","[KANDI TECHNOLOGIES GROUP INC, GEELY AUTOMOBIL...",en,"[(Kandi Technologies, (78, 96), kandi technolo...","[Kandi Technologies, Kandi]","[(78, 96), (146, 151)]"
3,![['ELectricCar News: Kandi's (NASDAQ GS: $KND...,"! Jinhua, China - July 25, 2016 (Newsfile Corp...",['MENA English (Middle East and North Africa F...,4514,2016-07-26,"July 26, 2016 Tuesday",NaN,Copyright 2016 MENAFN.COM All Rights Reserved,"[JOINT VENTURES, ELECTRIC VEHICLES, HOLDING CO...","[CHINA, NORTHERN AFRICA, MIDDLE EAST]","[DONGGUAN, GUANGDONG, CHINA]",[],"[NAICS336211 MOTOR VEHICLE BODY MANUFACTURING,...","[KANDI TECHNOLOGIES GROUP INC, GEELY AUTOMOBIL...",en,"[(Kandi Technologies, (78, 96), kandi technolo...","[Kandi Technologies, Kandi Electric Vehicles G...","[(78, 96), (175, 214)]"
3,![['ELectricCar News: Kandi's (NASDAQ GS: $KND...,"! Jinhua, China - July 25, 2016 (Newsfile Corp...",['MENA English (Middle East and North Africa F...,4514,2016-07-26,"July 26, 2016 Tuesday",NaN,Copyright 2016 MENAFN.COM All Rights Reserved,"[JOINT VENTURES, ELECTRIC VEHICLES, HOLDING CO...","[CHINA, NORTHERN AFRICA, MIDDLE EAST]","[DONGGUAN, GUANGDONG, CHINA]",[],"[NAICS336211 MOTOR VEHICLE BODY MANUFACTURING,...","[KANDI TECHNOLOGIES GROUP INC, GEELY AUTOMOBIL...",en,"[(Geely Automobile Holdings Ltd, (274, 303), g...","[Geely Automobile Holdings Ltd, GS]","[(274, 303), (117, 119)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493937,﻿Xanadu Mines (ASX:XAM) Acquisition of 90% of ...,"Feb 03, 2014 (The ABN Newswire: http://www.abn...",['The ABN Newswire'],9552,2014-02-05,"February 5, 2014 Wednesday",prf.admin,Copyright 2014 Newstex LLC All Rights Reserved,"[ALLIANCES & PARTNERSHIPS, JOINT VENTURES, PUB...",[MONGOLIA],"[ULAANBAATAR, MONGOLIA]",[],[NAICS212112 BITUMINOUS COAL UNDERGROUND MININ...,"[XANADU MINES LTD, TURQUOISE HILL RESOURCES LT...",en,"[(Turquoise Hill Resources Ltd, (286, 314), tu...","[Turquoise Hill Resources Ltd, THR]","[(286, 314), (317, 320)]"
1493942,﻿aaremaa dairy cooperative and E-Piim plan to ...,"Jan 08, 2014 (LETA (Latvia National News Agenc...",['LETA (Latvia National News Agency)'],1548,2014-01-12,"January 12, 2014 Sunday",prf.admin,Copyright 2014 Newstex LLC All Rights Rese

In [72]:
doc = df.sample()

text = doc.document.iloc[0]
spans = doc.spans.iloc[0]  # character-based entity spans 

inputs = tokenizer(text, entity_spans=spans, padding="max_length", truncation=True, return_tensors="pt",
                            max_length=MAX_LEN)
inputs.to(device)

outputs = model.model(**inputs)
logits = outputs.logits
predicted_classes = (torch.sigmoid(logits)>THRESHOLDS).float()
print("Sentence:", text)
print("Companies: ", doc.firms.iloc[0])
print("Tokens:", tokenizer.decode(inputs["input_ids"][0]))
print("Predicted class idx:", ids2labels(predicted_classes.squeeze().tolist()))
print("Confidence:", torch.sigmoid(logits).squeeze().tolist())

Sentence: Pactera Technology International Ltd. (NASDAQ: PACT) and Jatis Group said they have launched a new joint venture company Pactera-Jatis Indonesia. With operating approval obtained from Indonesia Investment Coordinating Board, the partnership establishes PJI as a leading global player in the region that builds upon the strengths of both companies to develop targeted industry solutions. The formation of PJI brings Pactera near-shore development capabilities in Indonesia to complement its global delivery centres located in mainland China. In addition to servicing customers in the APAC region, PJI is positioned to be a one-stop reliable centre for businesses around the globe seeking business expansion in South East Asia's booming economy. Jatis Group's key clientele includes leading APAC institutions in the financial, mobile-telecommunication, healthcare, manufacturing / distribution, fast moving consumer goods and public sectors. Headquartered in the Indonesian capital city of Ja

In [ ]:
# # model = LUKE.load_from_checkpoint(checkpoint_path=checkpoint_callback.best_model_path)
# model = LUKE.load_from_checkpoint(checkpoint_path=checkpoint_callback.best_model_path)
# model = model.to(device)

In [41]:
inf_dataset = RelationExtractionDataset(data=train_df)

In [43]:
from torch.utils.data import DataLoader

In [73]:
outputs = trainer.predict(dataloaders=DataLoader(inf_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=16))

TypeError: Unwrapping the module did not yield a `LightningModule`, got <class 'torch.utils.data.dataloader.DataLoader'> instead.

To deal with the imbalance, we use weights in the loss function proportional to the inverse relative frequency of each class, normalizing by the frequency of the most common class. The resulting weights are as follows:

In [ ]:
CLASS_WEIGHTS = 1/(rel_frequencies/rel_frequencies.iloc[0])
CLASS_WEIGHTS

The strategic alliance label is 6 times more frequent than the R&D label, therefore we weight the R&D class 6 times heavier in the loss function.

We can even increase the reweighting by exponentiating these weights:

In [ ]:
IMBALANCE_REWEIGHTING = 1
CLASS_WEIGHTS = CLASS_WEIGHTS**IMBALANCE_REWEIGHTING
CLASS_WEIGHTS

Now examples with the R&D label are weighted 10 times heavier than those with the SA label.

In [ ]:
CLASS_WEIGHTS = torch.tensor(CLASS_WEIGHTS.to_list()).to(device)